<a href="https://colab.research.google.com/github/fvgm-spec/learn-airbyte/blob/main/News_ETL_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Traditional ETL process

 The following is a traditional ETL process, which is based on batch data processing coming from API calls to [News API](https://newsapi.org/). The Data is collected and processed in predefined blocks as Python functions.

### Step 1: Importing required packages and setting global variables

In [ ]:
import pandas as pd
import json
import requests

In [ ]:
url = 'https://newsapi.org/v2/everything'
from google.colab import userdata
api_key = userdata.get('NEWS_API_KEY')

### Step 2: Defining functions

In [ ]:
#Defines a function to extract data from API endpoint
def api_data_extraction(api_endpoint: str, query: str, api_key: str):
    """
    Extracts data from an API endpoint

    Args:
        api_endpoint (str): name of the endpoint from the config.ini file.
        query (str): the search term or specific data to retrieve from API call.
        api_key (str): the API key required for authorization.
    Returns:
        dict: The JSON data extracted from the API call.
    """
    ##Extracting data from API
    r = requests.get(f"{api_endpoint}?q={query}&from=2024-03-21"
                     f"&sortBy=publishedAt&apiKey={api_key}")

    return r.json()

#Defines a function to transform JSON data into a pandas DataFrame
def data_transformation(json_data: dict) -> pd.DataFrame:
    """
    Converts the JSON data extracted from API call into a pandas DataFrame

    Args:
        json_data (dict): the data extracted from the API endpoint.
    Returns:
        pd.DataFrame: DataFrame containing the flattened data from the data in JSON format.
    """
    # Access the 'articles' list within the main dictionary
    articles = json_data['articles']

    # Create a list to store dictionaries with flattened data
    flat_data = []

    # Iterate through each article dictionary
    for article in articles:
        # Create a dictionary to store flattened key-value pairs
        article_data = {}

        # Access and store relevant information from the nested dictionary
        article_data['source_name'] = article['source']['name']
        article_data['author'] = article.get('author')
        article_data['title'] = article['title']
        article_data['description'] = article.get('description')
        article_data['url'] = article['url']
        article_data['urlToImage'] = article.get('urlToImage')
        article_data['publishedAt'] = article['publishedAt']

        # Append the flattened dictionary to the list
        flat_data.append(article_data)

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(flat_data)

    return df

### Step 3: Using helper functions to extract data from API endpoints and then performing data transformations

In [ ]:
data = api_data_extraction(url,"eclipse",api_key)

In [ ]:
df = data_transformation(data)

In [ ]:
# Transforming data extracted from API
df['publishedAt'] = pd.to_datetime(df['publishedAt'], utc=True)

# Create new columns for year, month, day
df['year'] = df['publishedAt'].dt.year
df['month'] = df['publishedAt'].dt.month_name()  # Month name as string
df['day'] = df['publishedAt'].dt.day

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   source_name  100 non-null    object             
 1   author       90 non-null     object             
 2   title        100 non-null    object             
 3   description  99 non-null     object             
 4   url          100 non-null    object             
 5   urlToImage   84 non-null     object             
 6   publishedAt  100 non-null    datetime64[ns, UTC]
 7   year         100 non-null    int32              
 8   month        100 non-null    object             
 9   day          100 non-null    int32              
dtypes: datetime64[ns, UTC](1), int32(2), object(7)
memory usage: 7.2+ KB


In [ ]:
df.head(5)

,source_name,author,title,description,url,urlToImage,publishedAt,year,month,day
0,Futurism,Victor Tangermann,Congresswoman Makes Strange Claims About the E...,"In an embarrassing slipup, representative Shei...",https://futurism.com/the-byte/congresswoman-st...,https://wp-assets.futurism.com/2024/04/congres...,2024-04-10 20:00:41+00:00,2024,April,10
1,Smithsonian.com,Sarah Kuta,When Are the Next Solar Eclipses? 2026 Promise...,The next total solar eclipse visible from the ...,https://www.smithsonianmag.com/smart-news/when...,https://th-thumbnailer.cdn-si-edu.com/bz2aHBfu...,2024-04-10 19:58:12+00:00,2024,April,10
2,Freerepublic.com,NY Post,Biden mocked for shuffling around White House ...,President Biden raised eyebrows Wednesday morn...,https://freerepublic.com/focus/f-news/4230338/...,None,2024-04-10 19:53:11+00:00,2024,April,10
3,Biztoc.com,cnbc.com,Sierra Space CEO unveils new satellite product...,Amid preparations for its spaceplane's maiden ...,https://biztoc.com/x/79629db2e2d30a1d,https://c.biztoc.com/p/79629db2e2d30a1d/s.webp,2024-04-10 19:50:06+00:00,2024,April,10
4,NASA,None,"More Than 36,000 Volunteers Helped Do NASA Ecl...","Thank you for helping us out! Over 36,000 peop...",https://science.nasa.gov/get-involved/citizen-...,https://science.nasa.gov/wp-content/uploads/20...,2024-04-10 19:47:05+00:00,2024,April,10


### Step 4

From this point we can proceed performing some other custom tranformation over the data and persisting the extracted data in a DBMS like Postgres, MySQL or DuckDB using SQLAlchemy library.